In [4]:
import pandas as pd

# Read the Parquet file
df = pd.read_parquet('output/orders_20251023_20251026.parquet')

# Show the first few rows
print(df.head())

# Show all rows (if only a few)
print(df)

# Show summary info
print(df.info())


   order_id order_date  user_id order_status  product_id  product_name  \
0     60001 2025-10-23     9047    completed        1024  Product_1024   
1     60003 2025-10-23     9042    completed        1025  Product_1025   
2     60006 2025-10-23     9040    completed        1006  Product_1006   
3     60014 2025-10-23     9047    completed        1009  Product_1009   
4     60017 2025-10-23     9020    completed        1007  Product_1007   

      category  qty  unit_price  order_total  stock_on_hand  \
0      Apparel    1      265.41       265.41            218   
1      Apparel    3      126.19       378.57             56   
2  Accessories    2       28.15        56.30            208   
3  Accessories    2      367.50       735.00            125   
4  Accessories    2      431.12       862.24            131   

          _source_file   price  
0  orders_20251023.csv  265.41  
1  orders_20251023.csv  126.19  
2  orders_20251023.csv   28.15  
3  orders_20251023.csv  367.50  
4  orders_2

In [2]:
import pandas as pd

orders = pd.read_parquet("C:\\Asan Innovators\\QuickShop-Assessment\\output\\orders_20251023_20251026.parquet")
orders = orders[orders['order_status'] == 'completed']

daily_total = orders.groupby('order_date')['order_total'].sum()
category_daily = orders.groupby(['order_date', 'category'])['order_total'].sum()

top_category = category_daily.groupby(level=0).idxmax()
top_category_revenue = category_daily.loc[top_category]

print(daily_total)
print(top_category_revenue)


order_date
2025-10-23    2353.82
2025-10-24    2068.90
2025-10-25    5316.67
2025-10-26    3780.27
Name: order_total, dtype: float64
order_date  category   
2025-10-23  Accessories    1709.84
2025-10-24  Electronics    1459.84
2025-10-25  Electronics    3571.00
2025-10-26  Home           1117.24
Name: order_total, dtype: float64


In [7]:
import pandas as pd
import sqlite3  # Use `import mysql.connector` for MySQL
from pathlib import Path

# --- For SQLite ---
db_path = Path("output/quickshop_etl.db")
conn = sqlite3.connect(db_path)

# --- For MySQL, replace with your connection ---
# import mysql.connector
# conn = mysql.connector.connect(
#     host="localhost",
#     user="your_user",
#     password="your_password",
#     database="quickshop_db"
# )

query = """
WITH first_orders AS (
    SELECT
        user_id,
        strftime('%Y-%m', MIN(order_date)) AS cohort_month
    FROM orders
    GROUP BY user_id
),
orders_with_cohort AS (
    SELECT
        o.user_id,
        f.cohort_month,
        strftime('%Y-%m', o.order_date) AS order_month
    FROM orders o
    JOIN first_orders f ON o.user_id = f.user_id
),
cohort_counts AS (
    SELECT
        cohort_month,
        order_month,
        COUNT(DISTINCT user_id) AS user_count
    FROM orders_with_cohort
    GROUP BY cohort_month, order_month
),
cohort_sizes AS (
    SELECT
        cohort_month,
        COUNT(DISTINCT user_id) AS cohort_size
    FROM first_orders
    GROUP BY cohort_month
)
SELECT
    c.cohort_month,
    c.order_month,
    c.user_count,
    cs.cohort_size,
    ROUND(c.user_count * 1.0 / cs.cohort_size, 2) AS retention_rate
FROM cohort_counts c
JOIN cohort_sizes cs ON c.cohort_month = cs.cohort_month
ORDER BY c.cohort_month, c.order_month;
"""

# Run query and load into DataFrame
df = pd.read_sql_query(query, conn)

print(df)

# Close connection
conn.close()


  cohort_month order_month  user_count  cohort_size  retention_rate
0      2025-10     2025-10          20           20             1.0
